# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst in order to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Data Analyst",
    "task": "Continuously monitor and analyze market data for AAPL",
    "context": "The task involves using statistical modeling and machine learning to identify trends and predict market movements for the selected stock AAPL."
}


> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL to analyze trends and predict market movements.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/AAPL"} 

Final Answer: 
Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, Mac, iPad, and various services like AppleCare support, cloud services, and platforms like the App Store. Apple Inc. serves consumers, small and mid-sized businesses, education, enterprise, and government markets. It distributes third-party applications through the App Store and sells its products through retail and online stores, direct sales, and third-party channels. Apple Inc. was founded in 1976 and is headquartered in Cupertino, California.

> Finished chain.
 

Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, Mac, iPad, and various services like AppleCare support, cloud services, and platforms like the App Store. Apple Inc. serves consumers, small and mid-sized businesses, education, enterprise, and government m

Final Answer:
After searching the internet for alternative sources of information to analyze market opportunities for AAPL, I found several valuable resources:

1. Title: Analyzing Porter's 5 Forces on Apple (AAPL) - Investopedia
Link: [Investopedia - Analyzing Porter's 5 Forces on Apple](https://www.investopedia.com/articles/investing/111015/analyzing-porters-five-forces-apple.asp)

2. Title: Apple Target Market Segmentation & Audience Analysis | Start.io
Link: [Start.io - Apple Target Market Segmentation & Audience Analysis](https://www.start.io/blog/apple-target-market-analysis-apple-demographics-marketing-strategy-competitors/)

3. Title: How Apple Uses Consumer Behavior Marketing to Win
Link: [Stephen Zoeller - Apple Consumer Behavior Marketing](https://www.stephenzoeller.com/apple-consumer-behavior-marketing/)

4. Title: Apple, Inc. (AAPL): Porter's Five Forces Industry and Competition Analysis - HiveIR
Link: [HiveIR - Apple Porter's Five Forces Analysis](https://www.hivelr.com/2

I need to gather more information from my team members to determine the best execution methods for AAPL based on the approved trading strategies.

Action: Ask question to co-worker
Action Input: {"coworker": "Data Analyst", "question": "Can you provide more insights on the trends and predictions for AAPL?", "context": "I need to understand the identified trends and predictions to implement a trend-following strategy for AAPL."}

> Entering new CrewAgentExecutor chain...
I need to gather real-time market data and insights on AAPL to provide accurate information for trend-following strategy implementation.

Action: Read website content
Action Input: {"website_url": "https://www.marketwatch.com/investing/stock/aapl"} 

marketwatch.comPlease enable JS and disable any ad blocker

Final Answer: marketwatch.comPlease enable JS and disable any ad blocker

> Finished chain.
 

marketwatch.comPlease enable JS and disable any ad blocker

Thought: 
I have gathered insights on the trends and predic

Final Answer: 
Based on the analysis of Apple Inc.'s (AAPL) current market dynamics and the proposed trading strategies of Trend Following Strategy, Options Trading Strategy, Swing Trading Strategy, Fundamental Analysis Strategy, and Risk Management Strategy, here are the potential risks and suggested mitigation strategies:

1. Trend Following Strategy: The potential risk with this strategy is that trends can reverse suddenly, leading to losses. To mitigate this risk, it is essential to set strict stop-loss orders and regularly monitor the trend indicators to exit trades promptly.

2. Options Trading Strategy: Options trading carries inherent risks such as time decay and volatility. To mitigate these risks, it is crucial to diversify the options portfolio, use risk management techniques like hedging, and stay informed about market news and events that can impact options prices.

3. Swing Trading Strategy: The risk in swing trading lies in holding positions overnight, exposing them to o

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Based on the analysis of Apple Inc.'s (AAPL) current market dynamics and the proposed trading strategies of Trend Following Strategy, Options Trading Strategy, Swing Trading Strategy, Fundamental Analysis Strategy, and Risk Management Strategy, here are the potential risks and suggested mitigation strategies:

1. Trend Following Strategy: The potential risk with this strategy is that trends can reverse suddenly, leading to losses. To mitigate this risk, it is essential to set strict stop-loss orders and regularly monitor the trend indicators to exit trades promptly.

2. Options Trading Strategy: Options trading carries inherent risks such as time decay and volatility. To mitigate these risks, it is crucial to diversify the options portfolio, use risk management techniques like hedging, and stay informed about market news and events that can impact options prices.

3. Swing Trading Strategy: The risk in swing trading lies in holding positions overnight, exposing them to overnight gaps or market volatility. To mitigate this risk, consider using smaller position sizes, setting tight stop-loss orders, and avoiding holding positions through major news events.

4. Fundamental Analysis Strategy: Risks associated with fundamental analysis include misinterpretation of data or unexpected market reactions to fundamental news. To mitigate these risks, ensure thorough research, use multiple data sources, and consider technical analysis signals for confirmation.

5. Risk Management Strategy: The risk management strategy itself carries the risk of overcompensating for potential losses, leading to missed opportunities. To mitigate this risk, maintain a balanced approach to risk management, set realistic risk-reward ratios, and regularly review and adjust risk tolerance levels.

By considering these potential risks and implementing the suggested mitigation strategies, the trading activities for AAPL can align with the firm's risk tolerance and potentially enhance overall trading performance.